# Instalación:


### Las primeras 3 celdas solo se ejecutan 1 vez




In [ ]:
import os, sys
from google.colab import drive

drive.mount('/content/drive')
nb_path = '/content/notebooks'
os.symlink('/content/drive/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0,nb_path)

Mounted at /content/drive


## Instalación de biblioteca Stable Baseline



### Biblioteca que contiene los modelos


In [ ]:
!pip install --target=$nb_path git+https://github.com/Stable-Baselines-Team/stable-baselines3-contrib

  Cloning https://github.com/Stable-Baselines-Team/stable-baselines3-contrib to /tmp/pip-req-build-4zonuzzl
  Running command git clone --filter=blob:none --quiet https://github.com/Stable-Baselines-Team/stable-baselines3-contrib /tmp/pip-req-build-4zonuzzl
  Resolved https://github.com/Stable-Baselines-Team/stable-baselines3-contrib to commit 707cb0f9cd9104c989a3e4d4cf30965b64ca7339
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached stable_baselines3-2.2.1-py3-none-any.whl (181 kB)
  Using cached gymnasium-0.29.1-py3-none-any.whl (953 kB)
  Using cached Farama_Notifications-0.0.4-py3-none-any.whl (2.5 kB)
  Created wheel for sb3-contrib: filename=sb3_contrib-2.2.1-py3-none-any.whl size=80668 sha256=5916a69122ff1d5b70b89c2bac2bdbdd7175d9ca11e0326604f1e4326bac5b9b
  Stored in directory: /tmp/pip-ephem-wheel-cache-xhqkjy67/wheels/f8/e1/97/7e886248901c65ffe4ff7a9c16bccfbde992692ac10d99ee41
S

## Instalación del Dream On Gym

### Contiene el ambiente de redes opticas y el simulador

In [ ]:
!pip install --target=$nb_path dream-on-gym-v2==0.0.6

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 56.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 75.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 84.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 38.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 77.5 MB/s eta 0:00:00
  Created wheel for dream-on-gym-v2: filename=dream_on_gym_v2-0.0.6-py3-none-any.whl size=29791 sha256=b9e813d6aba062974ff059



---

# Codigo



In [ ]:
import os
import sys
from google.colab import drive
drive.mount('/content/drive')
sys.path.append('/content/drive/My Drive/Colab Notebooks')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import torch as th

from sb3_contrib import TRPO
from sb3_contrib.trpo import MlpPolicy as MLP_TRPO

from dreamongymv2.simNetPy import *
from dreamongymv2.gym_basic import *
import gymnasium

sys.modules["gym"] = gymnasium

In [ ]:
bands = ['L','C','S','E']

episodeRewardAccum = 0
iteration = 0
episodeIteration = 100
episodeRewardsArray = []

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
def getBand(actionBand):
    if (actionBand == 0):
        return 'C'
    elif (actionBand == 1):
        return 'L'
    elif (actionBand == 2):
        return 'S'
    elif (actionBand == 3):
        return 'E'

    return 'NoBand'

In [ ]:
def reward():
    global episodeRewardAccum
    global episodeIteration
    global episodeRewardsArray
    global iteration

    iteration = iteration + 1
    if (iteration % episodeIteration) == 0:
        episodeRewardsArray.append(episodeRewardAccum)
        episodeRewardAccum = 0

    value = env.getSimulator().lastConnectionIsAllocated()
    if (value.name == Controller.Status.Not_Allocated.name):
        #print(value.name)
        value = -1
    else:
        value = 1
    episodeRewardAccum = episodeRewardAccum + value
    return value

In [ ]:
def state():
    connectionEvent = env.getSimulator().connectionEvent
    route = env.getSimulator().controller.path[connectionEvent.source][connectionEvent.destination][0]
    slotsQuantity = 0
    for i in range(4):
        slotsQuantity = slotsQuantity + route[0].getSlots(getBand(i))
    obs = [0] * slotsQuantity
    slotInit = 0
    for i in range(len(route)):
        slotIterator = 0
        for j in range(4):
            slotInit = slotIterator
            for k in range(slotInit, slotInit + route[i].getSlots(getBand(j))):
                slotIterator = slotIterator + 1
                if route[i].getSlot(k-slotInit,getBand(j)):
                    obs[k] = obs[k] + 1
    return obs


In [ ]:
# def stateAllNetwork():
#     connectionEvent = env.getSimulator().connectionEvent
#     routes = env.getSimulator().controller.path[connectionEvent.source][connectionEvent.destination]
#     slotsQuantity = 0
#     for i in range(4):
#         slotsQuantity = slotsQuantity + routes[0][0].getSlots(getBand(i))
#     obs = [0] * slotsQuantity
#     slotInit = 0
#     for i in range(len(env.getSimulator().controller.network.links)):
#         slotIterator = 0
#         for z in range(4):
#             slotInit = slotIterator
#             for k in range(slotInit, slotInit + env.getSimulator().controller.network.links[i].getSlots(getBand(z))):
#                 slotIterator = slotIterator + 1
#                 if env.getSimulator().controller.network.links[i].getSlot(k-slotInit,getBand(z)):
#                     obs[k] = obs[k] + 1
#     return obs

In [ ]:
def first_fit_algorithm(src: int, dst: int, b: BitRate, c: Connection, n: Network, path, action):
    numberOfSlots = b.getNumberofSlots(0)
    link_ids = path[src][dst][0]
    c.bandSelected = "C"
    general_link = []
    for _ in range(n.getLink(0).getSlots(c.bandSelected)):
        general_link.append(False)

    for link in link_ids:
        link = n.getLink(link.id)
        link.bandSelected = c.bandSelected
        for slot in range(link.getSlots(c.bandSelected)):
            general_link[slot] = general_link[slot] or link.getSlot(slot,c.bandSelected)
    currentNumberSlots = 0
    currentSlotIndex = 0
    for j in range(len(general_link)):
        if not general_link[j]:
            currentNumberSlots += 1
        else:
            currentNumberSlots = 0
            currentSlotIndex = j + 1
        if currentNumberSlots == numberOfSlots:
            for k in link_ids:
                c.addLink(
                    k, fromSlot=currentSlotIndex, toSlot=currentSlotIndex+currentNumberSlots)
            return Controller.Status.Allocated, c
    return Controller.Status.Not_Allocated, c

In [ ]:
def agent_algorithm(src: int, dst: int, b: BitRate, c: Connection, n: Network, path, action):
    band = getBand(action[0])
    c.bandSelected = band
    numberOfSlots = b.getNumberofSlots(0)
    actionSpace = len(path[src][dst])
    link_ids = path[src][dst][0]
    general_link = []
    for _ in range(n.getLink(0).getSlots(band)):
        general_link.append(False)
    for link in link_ids:
        link = n.getLink(link.id)
        link.bandSelected = band
        for slot in range(link.getSlots()):
            general_link[slot] = general_link[slot] or link.getSlot(slot,band)
    currentNumberSlots = 0
    currentSlotIndex = 0
    for j in range(len(general_link)):
        if not general_link[j]:
            currentNumberSlots += 1
        else:
            currentNumberSlots = 0
            currentSlotIndex = j + 1
        if currentNumberSlots == numberOfSlots:
            for k in link_ids:
                c.addLink(
                    k, fromSlot=currentSlotIndex, toSlot=currentSlotIndex+currentNumberSlots)
            return Controller.Status.Allocated, c
    return Controller.Status.Not_Allocated, c

In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.ppo.policies import MlpPolicy

In [ ]:
fileDirectory = '/content/drive/MyDrive/DREAMONGYM'

env = gymnasium.make("rlonenv-v0")

obs, info = env.reset()

#definir espacio (opciones del 0-3, 4 en total )
env.action_space = gymnasium.spaces.MultiDiscrete(np.array([4]))


#tamaño de los slots de las bandas
env.observation_space = gymnasium.spaces.MultiDiscrete([100] * 2720)

#Set funcion recompensa
env.setRewardFunc(reward)
#Set estados
env.setStateFunc(state)
#env.setStateFunc(stateAllNetwork)
#Definir topologia y rutas
env.initEnviroment(fileDirectory + "/Topologias-rutas/Eurocore_4_bands.json", fileDirectory + "/Topologias-rutas/Eurocore_routes.json")
#Cantidad de conexiones para tener el simulador corriendo
env.getSimulator().goalConnections = 100000
env.getSimulator().setMu(1)
env.getSimulator().setLambda(1000)
env.getSimulator().setAllocator(first_fit_algorithm)
env.getSimulator().init()

/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.setRewardFunc to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.setRewardFunc` for environment variables or `env.get_wrapper_attr('setRewardFunc')` that will search the reminding wrappers.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.setStateFunc to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.setStateFunc` for environment variables or `env.get_wrapper_attr('setStateFunc')` that will search the reminding wrappers.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.initEnviroment to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.initEnviroment` for environment variables

meta: 0.95
objetivo: 0.95 covered: 0.0 epsilon: 0.01
meta: 0.2673105078629141
objetivo: 0.95 covered: 0.6826894921370859 epsilon: 0.01


/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.getSimulator to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.getSimulator` for environment variables or `env.get_wrapper_attr('getSimulator')` that will search the reminding wrappers.
  logger.warn(


#Entrenamiento del modelo


In [ ]:
env.start()
env.getSimulator().setAllocator(agent_algorithm)

policy_args = dict(net_arch=5*[128], activation_fn=th.nn.ReLU)


model = PPO(MlpPolicy, env, verbose=0, tensorboard_log="./tb/PPO-DeepRMSA-v0/", policy_kwargs=policy_args, gamma=.95)

model.learn(total_timesteps=1000, log_interval=4)

modelDirectory = fileDirectory + "/models/"

model.save(modelDirectory+ "PPO-Eurocore-RW1")
#model.load('TRPO100720231300')

/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.start to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.start` for environment variables or `env.get_wrapper_attr('start')` that will search the reminding wrappers.
  logger.warn(


Nodes: 11
Links: 50
Goal Connections: 100000
Lambda: 1000
Mu: 1
+----------+----------+----------+----------+----------+----------+----------+
| progress |  arrives | blocking |  time(s) |  Wald CI |  A-C. CI |Wilson CI |
+----------+----------+----------+----------+----------+----------+----------+
|     5.0% |     5000 | 0.357329 |  0:00:01 | 0.013553 | 0.013548 | 0.013548 |
+----------+----------+----------+----------+----------+----------+----------+
|    10.0% |    10000 | 0.363364 |  0:00:04 | 0.009619 | 0.009617 | 0.009617 |
+----------+----------+----------+----------+----------+----------+----------+
|    15.0% |    15000 | 0.357976 |  0:00:07 | 0.007828 | 0.007828 | 0.007827 |
+----------+----------+----------+----------+----------+----------+----------+
|    20.0% |    20000 | 0.357182 |  0:00:09 | 0.006776 | 0.006776 | 0.006776 |
+----------+----------+----------+----------+----------+----------+----------+
|    25.0% |    25000 | 0.353986 |  0:00:11 | 0.006049 | 0.006048 |

/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.getSimulator to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.getSimulator` for environment variables or `env.get_wrapper_attr('getSimulator')` that will search the reminding wrappers.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gymnasium/utils/passive_env_checker.py:127: UserWarning: WARN: The obs returned by the `step()` method should be an int or np.int64, actual type: <class 'list'>
  logger.warn(f"{pre} should be an int or np.int64, actual type: {type(obs)}")
/usr/local/lib/python3.10/dist-packages/gymnasium/utils/passive_env_checker.py:159: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/save_util.py:283: UserWarning: Path '/content/driv

# Ejecución del modelo

In [ ]:
obs, info = env.reset()
probBloqueoArray = []
rewardsArray = []
rewardTemp = 0
NoAlloCounter = 0
ConnCounter = 1

iterationAccum = 100
_state = [0] * 2720
for i in range(100000):
    if (i % iterationAccum) == 0:
        probBloqueoArray.append(NoAlloCounter/ConnCounter)
        rewardsArray.append(rewardTemp)
        rewardTemp = 0
    action, _states = model.predict(_state)
    _state, _reward, terminated, truncated, info = env.step(action)
    rewardTemp = rewardTemp + _reward
    if _reward == -1:
      NoAlloCounter += 1
      ConnCounter += 1
    else:
      ConnCounter += 1

print(episodeRewardsArray)
print(rewardsArray)
print(probBloqueoArray)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[87, 96, 94, 88, 90, 90, 90, 98, 92, 88, 90, 82, 80, 88, 92, 86, 92, 92, 90, 86, 92, 88, 92, 82, 88, 84, 90, 90, 84, 90, 96, 90, 96, 94, 90, 82, 100, 98, 96, 84, 98, 90, 92, 86, 90, 94, 92, 88, 90, 92, 90, 96, 86, 92, 92, 82, 92, 88, 82, 78, 90, 84, 90, 90, 84, 92, 84, 96, 94, 96, 96, 88, 92, 88, 82, 90, 92, 88, 90, 80, 88, 88, 86, 92, 86, 82, 90, 90, 90, 90, 86, 90, 88, 82, 78, 78, 86, 80, 90, 86, 94, 90, 96, 94, 92, 84, 82, 98, 90, 80, 98, 88, 90, 90, 96, 88, 92, 86, 88, 88, 80, 92, 92, 80, 98, 88, 90, 92, 86, 92, 86, 88, 94, 84, 92, 90, 84, 90, 88, 90, 94, 88, 88, 78, 82, 68, 92, 88, 92, 88, 94, 90, 88, 86, 90, 88, 86, 94, 88, 94, 88, 94, 92, 86, 88, 96, 88, 94, 86, 92, 78, 88, 90, 90, 88, 88, 88, 90, 86, 92, 90, 84, 82, 94, 80, 96, 88, 90, 84, 90, 96, 86, 90, 92, 94, 82, 90, 86, 82, 86, 90, 90, 94, 88, 92, 84, 92, 98, 96, 94, 92, 86, 86, 88, 84, 82, 92, 96, 90, 84, 86, 88, 88, 92, 88, 90, 94, 84, 86, 94, 100, 92, 92, 96, 90, 90, 98, 84, 94, 94, 90, 86, 92, 90, 90, 94, 88, 100, 86, 

In [ ]:
fileDirectory = '/content/drive/MyDrive/DREAMONGYM'

env = gymnasium.make("rlonenv-v0")

obs, info = env.reset()

#definir espacio (opciones del 0-3, 4 en total )
env.action_space = gymnasium.spaces.MultiDiscrete(np.array([4]))


#tamaño de los slots de las bandas
env.observation_space = gymnasium.spaces.MultiDiscrete([100] * 2720)

#Set funcion recompensa
env.setRewardFunc(reward)
#Set estados
env.setStateFunc(state)
#env.setStateFunc(stateAllNetwork)
#Definir topologia y rutas
env.initEnviroment(fileDirectory + "/Topologias-rutas/NSFNet_4_bands.json", fileDirectory + "/Topologias-rutas/NSFNet_routes.json")
#Cantidad de conexiones para tener el simulador corriendo
env.getSimulator().goalConnections = 100000
env.getSimulator().setMu(1)
env.getSimulator().setLambda(1000)
env.getSimulator().setAllocator(first_fit_algorithm)
env.getSimulator().init()

/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.setRewardFunc to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.setRewardFunc` for environment variables or `env.get_wrapper_attr('setRewardFunc')` that will search the reminding wrappers.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.setStateFunc to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.setStateFunc` for environment variables or `env.get_wrapper_attr('setStateFunc')` that will search the reminding wrappers.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.initEnviroment to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.initEnviroment` for environment variables

meta: 0.95
objetivo: 0.95 covered: 0.0 epsilon: 0.01
meta: 0.2673105078629141
objetivo: 0.95 covered: 0.6826894921370859 epsilon: 0.01


In [ ]:
env.start()
env.getSimulator().setAllocator(agent_algorithm)

policy_args = dict(net_arch=5*[128], activation_fn=th.nn.ReLU)


model = PPO(MlpPolicy, env, verbose=0, tensorboard_log="./tb/PPO-DeepRMSA-v0/", policy_kwargs=policy_args, gamma=.95)

model.learn(total_timesteps=1000, log_interval=4)

modelDirectory = fileDirectory + "/models/"

model.save(modelDirectory+ "PPO-NSFNet-RW1")
#model.load('TRPO100720231300')

/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.start to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.start` for environment variables or `env.get_wrapper_attr('start')` that will search the reminding wrappers.
  logger.warn(


Nodes: 14
Links: 44
Goal Connections: 100000
Lambda: 1000
Mu: 1
+----------+----------+----------+----------+----------+----------+----------+
| progress |  arrives | blocking |  time(s) |  Wald CI |  A-C. CI |Wilson CI |
+----------+----------+----------+----------+----------+----------+----------+
|     5.0% |     5000 | 0.418316 |  0:00:02 | 0.013951 | 0.013945 | 0.013945 |
+----------+----------+----------+----------+----------+----------+----------+
|    10.0% |    10000 | 0.425957 |  0:00:04 | 0.009889 | 0.009887 | 0.009887 |
+----------+----------+----------+----------+----------+----------+----------+
|    15.0% |    15000 | 0.426772 |  0:00:06 | 0.008077 | 0.008076 | 0.008076 |
+----------+----------+----------+----------+----------+----------+----------+
|    20.0% |    20000 | 0.424179 |  0:00:08 | 0.006989 | 0.006988 | 0.006988 |
+----------+----------+----------+----------+----------+----------+----------+
|    25.0% |    25000 | 0.422743 |  0:00:10 | 0.006248 | 0.006248 |

In [ ]:
obs, info = env.reset()
probBloqueoArray = []
rewardsArray = []
rewardTemp = 0
NoAlloCounter = 0
ConnCounter = 1

iterationAccum = 100
_state = [0] * 2720
for i in range(100000):
    if (i % iterationAccum) == 0:
        probBloqueoArray.append(NoAlloCounter/ConnCounter)
        rewardsArray.append(rewardTemp)
        rewardTemp = 0
    action, _states = model.predict(_state)
    _state, _reward, terminated, truncated, info = env.step(action)
    rewardTemp = rewardTemp + _reward
    if _reward == -1:
      NoAlloCounter += 1
      ConnCounter += 1
    else:
      ConnCounter += 1

print(episodeRewardsArray)
print(rewardsArray)
print(probBloqueoArray)

[87, 96, 94, 88, 90, 90, 90, 98, 92, 88, 90, 82, 80, 88, 92, 86, 92, 92, 90, 86, 92, 88, 92, 82, 88, 84, 90, 90, 84, 90, 96, 90, 96, 94, 90, 82, 100, 98, 96, 84, 98, 90, 92, 86, 90, 94, 92, 88, 90, 92, 90, 96, 86, 92, 92, 82, 92, 88, 82, 78, 90, 84, 90, 90, 84, 92, 84, 96, 94, 96, 96, 88, 92, 88, 82, 90, 92, 88, 90, 80, 88, 88, 86, 92, 86, 82, 90, 90, 90, 90, 86, 90, 88, 82, 78, 78, 86, 80, 90, 86, 94, 90, 96, 94, 92, 84, 82, 98, 90, 80, 98, 88, 90, 90, 96, 88, 92, 86, 88, 88, 80, 92, 92, 80, 98, 88, 90, 92, 86, 92, 86, 88, 94, 84, 92, 90, 84, 90, 88, 90, 94, 88, 88, 78, 82, 68, 92, 88, 92, 88, 94, 90, 88, 86, 90, 88, 86, 94, 88, 94, 88, 94, 92, 86, 88, 96, 88, 94, 86, 92, 78, 88, 90, 90, 88, 88, 88, 90, 86, 92, 90, 84, 82, 94, 80, 96, 88, 90, 84, 90, 96, 86, 90, 92, 94, 82, 90, 86, 82, 86, 90, 90, 94, 88, 92, 84, 92, 98, 96, 94, 92, 86, 86, 88, 84, 82, 92, 96, 90, 84, 86, 88, 88, 92, 88, 90, 94, 84, 86, 94, 100, 92, 92, 96, 90, 90, 98, 84, 94, 94, 90, 86, 92, 90, 90, 94, 88, 100, 86, 

In [ ]:
def rewardBandSelectShorter(): ########### This one
    global episodeRewardAccum
    global episodeIteration
    global episodeRewardsArray
    global iteration

    iteration = iteration + 1
    if (iteration % episodeIteration) == 0:
        episodeRewardsArray.append(episodeRewardAccum)
        episodeRewardAccum = 0

    reward = 0
    connectionsNow = env.getSimulator().controller.connections

    value = env.getSimulator().lastConnectionIsAllocated()
    if (value.name == Controller.Status.Not_Allocated.name):
        reward -= 1.0
    elif connectionsNow[-1].bandSelected == "C" or connectionsNow[-1].bandSelected == "L" or connectionsNow[-1].bandSelected == "S":
        reward +=1.0
    elif connectionsNow[-1].bandSelected == "E":
        reward +=0.8

    episodeRewardAccum = episodeRewardAccum + reward
    return reward

In [ ]:
fileDirectory = '/content/drive/MyDrive/DREAMONGYM'

env = gymnasium.make("rlonenv-v0")

obs, info = env.reset()

#definir espacio (opciones del 0-3, 4 en total )
env.action_space = gymnasium.spaces.MultiDiscrete(np.array([4]))


#tamaño de los slots de las bandas
env.observation_space = gymnasium.spaces.MultiDiscrete([100] * 2720)

#Set funcion recompensa
env.setRewardFunc(rewardBandSelectShorter)
#Set estados
env.setStateFunc(state)
#env.setStateFunc(stateAllNetwork)
#Definir topologia y rutas
env.initEnviroment(fileDirectory + "/Topologias-rutas/NSFNet_4_bands.json", fileDirectory + "/Topologias-rutas/NSFNet_routes.json")
#Cantidad de conexiones para tener el simulador corriendo
env.getSimulator().goalConnections = 100000
env.getSimulator().setMu(1)
env.getSimulator().setLambda(1000)
env.getSimulator().setAllocator(first_fit_algorithm)
env.getSimulator().init()

meta: 0.95
objetivo: 0.95 covered: 0.0 epsilon: 0.01
meta: 0.2673105078629141
objetivo: 0.95 covered: 0.6826894921370859 epsilon: 0.01


In [ ]:
env.start()
env.getSimulator().setAllocator(agent_algorithm)

policy_args = dict(net_arch=5*[128], activation_fn=th.nn.ReLU)


model = PPO(MlpPolicy, env, verbose=0, tensorboard_log="./tb/PPO-DeepRMSA-v0/", policy_kwargs=policy_args, gamma=.95)

model.learn(total_timesteps=1000, log_interval=4)

modelDirectory = fileDirectory + "/models/"

model.save(modelDirectory+ "PPO-NSFNet-RW2")
#model.load('TRPO100720231300')

Nodes: 14
Links: 44
Goal Connections: 100000
Lambda: 1000
Mu: 1
+----------+----------+----------+----------+----------+----------+----------+
| progress |  arrives | blocking |  time(s) |  Wald CI |  A-C. CI |Wilson CI |
+----------+----------+----------+----------+----------+----------+----------+
|     5.0% |     5000 | 0.418316 |  0:00:01 | 0.013951 | 0.013945 | 0.013945 |
+----------+----------+----------+----------+----------+----------+----------+
|    10.0% |    10000 | 0.425957 |  0:00:04 | 0.009889 | 0.009887 | 0.009887 |
+----------+----------+----------+----------+----------+----------+----------+
|    15.0% |    15000 | 0.426772 |  0:00:07 | 0.008077 | 0.008076 | 0.008076 |
+----------+----------+----------+----------+----------+----------+----------+
|    20.0% |    20000 | 0.424179 |  0:00:09 | 0.006989 | 0.006988 | 0.006988 |
+----------+----------+----------+----------+----------+----------+----------+
|    25.0% |    25000 | 0.422743 |  0:00:11 | 0.006248 | 0.006248 |

In [ ]:
obs, info = env.reset()
probBloqueoArray = []
rewardsArray = []
rewardTemp = 0
NoAlloCounter = 0
ConnCounter = 1

iterationAccum = 100
_state = [0] * 2720
for i in range(100000):
    if (i % iterationAccum) == 0:
        probBloqueoArray.append(NoAlloCounter/ConnCounter)
        rewardsArray.append(rewardTemp)
        rewardTemp = 0
    action, _states = model.predict(_state)
    _state, _reward, terminated, truncated, info = env.step(action)
    rewardTemp = rewardTemp + _reward
    if _reward == -1:
      NoAlloCounter += 1
      ConnCounter += 1
    else:
      ConnCounter += 1

print(episodeRewardsArray)
print(rewardsArray)
print(probBloqueoArray)

[87, 96, 94, 88, 90, 90, 90, 98, 92, 88, 90, 82, 80, 88, 92, 86, 92, 92, 90, 86, 92, 88, 92, 82, 88, 84, 90, 90, 84, 90, 96, 90, 96, 94, 90, 82, 100, 98, 96, 84, 98, 90, 92, 86, 90, 94, 92, 88, 90, 92, 90, 96, 86, 92, 92, 82, 92, 88, 82, 78, 90, 84, 90, 90, 84, 92, 84, 96, 94, 96, 96, 88, 92, 88, 82, 90, 92, 88, 90, 80, 88, 88, 86, 92, 86, 82, 90, 90, 90, 90, 86, 90, 88, 82, 78, 78, 86, 80, 90, 86, 94, 90, 96, 94, 92, 84, 82, 98, 90, 80, 98, 88, 90, 90, 96, 88, 92, 86, 88, 88, 80, 92, 92, 80, 98, 88, 90, 92, 86, 92, 86, 88, 94, 84, 92, 90, 84, 90, 88, 90, 94, 88, 88, 78, 82, 68, 92, 88, 92, 88, 94, 90, 88, 86, 90, 88, 86, 94, 88, 94, 88, 94, 92, 86, 88, 96, 88, 94, 86, 92, 78, 88, 90, 90, 88, 88, 88, 90, 86, 92, 90, 84, 82, 94, 80, 96, 88, 90, 84, 90, 96, 86, 90, 92, 94, 82, 90, 86, 82, 86, 90, 90, 94, 88, 92, 84, 92, 98, 96, 94, 92, 86, 86, 88, 84, 82, 92, 96, 90, 84, 86, 88, 88, 92, 88, 90, 94, 84, 86, 94, 100, 92, 92, 96, 90, 90, 98, 84, 94, 94, 90, 86, 92, 90, 90, 94, 88, 100, 86, 